今回は、feedforward neural networkを使って、固定ウィンドウ幅(左右３)の情報を使って単語の品詞タグを予測するモデルを学習してみます．  
かなりシンプルですが、高速に予測できるという魅力から、より強力なモデルが沢山存在する今でも、NLPのいろんなツールで使われてます．  
easyccg: https://github.com/mikelewis0/easyccg  
syntaxnet: https://github.com/tensorflow/models/tree/master/research/syntaxnet  
stanford parser: https://nlp.stanford.edu/software/lex-parser.shtml  
(イメージ)　　
<img src='images/ff.png'>
(画像: https://xbt.net/blog/what-is-enigma/)

POSタグ(Part-Of-Speech, 品詞タグ)のリスト:  
- ADJ: adjective
- ADP: adposition
- ADV: adverb
- AUX: auxiliary
- CCONJ: coordinating conjunction
- DET: determiner
- INTJ: interjection
- NOUN: noun
- NUM: numeral
- PART: particle
- PRON: pronoun
- PROPN: proper noun
- PUNCT: punctuation
- SCONJ: subordinating conjunction
- SYM: symbol
- VERB: verb
- X: other

In [1]:
# つかうライブラリの読み込み

import numpy as np
import keras
from keras.models import Sequential
from keras.layers import Dense, Dropout, Embedding, Input, Reshape, Concatenate
from keras.optimizers import SGD
from collections import Counter

# 👇無視してOK

Using TensorFlow backend.


In [2]:
import tensorflow as tf
from keras.backend.tensorflow_backend import set_session
config = tf.ConfigProto(
    gpu_options=tf.GPUOptions(
        visible_device_list="0", # specify GPU number
        allow_growth=True
    )
)
set_session(tf.Session(config=config))

In [3]:
# CoNLLフォーマットを読み込む関数
def read_conll(file):
    res = []
    words = []
    tags = []
    for line in open(file):
        line = line.strip()
         # 空行ならそれまでにつくった文を出力
        if len(line) == 0:
            res.append((words, tags))
            words = []
            tags = []
        # 単語とタグを取り出す
        else:
            items = line.split('\t')
            words.append(items[1].lower()) # 小文字にしておく
            tags.append(items[3])
    return res

In [4]:
# 学習データ (training data)
train_sents = read_conll('data/train.conll')
# 評価用データ (test data)
test_sents = read_conll('data/test.conll')
# 開発データ (development data)
dev_sents = read_conll('data/dev.conll')

In [5]:
def get_prefix(word):
    word = word.rjust(2, ' ')
    return word[:2]

def get_suffix(word):
    word = word.ljust(2, ' ')
    return word[-2:]

In [6]:
# テスト
word = 'dog'
print(get_prefix(word), get_suffix(word))
word = 'is'
print(get_prefix(word), get_suffix(word))
word = 'a'
print(get_prefix(word), get_suffix(word))

do og
is is
 a a 


In [7]:
def sliding_windows(lst):
    res = []
    for i in range(len(lst) - 6):
        res.append(lst[i:i+7])
    return res

In [8]:
PAD = 'PAD'

train_sents = [([PAD] * 3 + words + [PAD] * 3, tags) for words, tags in train_sents]
test_sents = [([PAD] * 3 + words + [PAD] * 3, tags) for words, tags in test_sents]
dev_sents = [([PAD] * 3 + words + [PAD] * 3, tags) for words, tags in dev_sents]

In [9]:
def make_vocab(tokens, max_count=2, UNK='UNK'):
    counter = Counter(tokens)
    token_list = [token for token, count in counter.most_common() if count >= max_count]
    token_list.append(UNK)
    token_dict = {t: i for i, t in enumerate(token_list)}
    return token_dict

In [10]:
UNK = 'UNK'
# 単語の埋め込みベクトルがうまくいくためには、その単語がいろいろな文脈で出現してほしい．
# 学習データにちょっと(２回より下)しか出ない単語はUNKで置き換える．
word_dict = make_vocab(word for words, _ in train_sents for word in words)
prefix_dict = make_vocab(get_prefix(word) for words, _ in train_sents for word in words)
suffix_dict = make_vocab(get_suffix(word) for words, _ in train_sents for word in words)

In [11]:
#POSタグを自然数のIDに変換する辞書
tag_set = set(tag for _, tags in train_sents for tag in tags)
tag_dict = {w: i for i, w in enumerate(tag_set)}

In [12]:
print('word_dict size', len(word_dict))
print('prefix_dict size', len(prefix_dict))
print('suffix_dict size', len(suffix_dict))
print('tag_dict size', len(tag_dict))

word_dict size 21569
prefix_dict size 671
suffix_dict size 662
tag_dict size 17


In [13]:
def make_matrices(words_and_tags):
    word_xs = []
    prefix_xs = []
    suffix_xs = []
    ys = []
    for words, tags in words_and_tags:
        for window in sliding_windows(words):
            word_xs.append([word_dict.get(word, word_dict[UNK]) for word in window])
            prefix_xs.append([prefix_dict.get(get_prefix(word), prefix_dict[UNK]) for word in window])
            suffix_xs.append([suffix_dict.get(get_suffix(word), suffix_dict[UNK]) for word in window])
        ys.extend(tag_dict[tag] for tag in tags)

    word_xs = np.array(word_xs, 'i')
    prefix_xs = np.array(prefix_xs, 'i')
    suffix_xs = np.array(suffix_xs, 'i')
    ys = np.array(ys, 'i')
    ys = keras.utils.to_categorical(ys, len(tag_dict))
    print('dimensions of xs', word_xs.shape, prefix_xs.shape, suffix_xs.shape)
    print('dimensions of ys', ys.shape)
    return [word_xs, prefix_xs, suffix_xs], ys

In [14]:
train_xs, train_ys = make_matrices(train_sents)
test_xs, test_ys = make_matrices(test_sents)
dev_xs, dev_ys = make_matrices(dev_sents)

dimensions of xs (929552, 7) (929552, 7) (929552, 7)
dimensions of ys (929552, 17)
dimensions of xs (55371, 7) (55371, 7) (55371, 7)
dimensions of ys (55371, 17)
dimensions of xs (45422, 7) (45422, 7) (45422, 7)
dimensions of ys (45422, 17)


In [15]:
def make_model(word_input, prefix_input, suffix_input, word_vocab_size, prefix_vocab_size, suffix_vocab_size, n_tags,
              WORD_EMBED_DIM=128, PREFIX_EMBED_DIM=32, SUFFIX_EMBED_DIM=32, HIDDEN1_DIM=256, HIDDEN2_DIM=128):
    word_embed = Embedding(word_vocab_size, WORD_EMBED_DIM)(word_input)
    prefix_embed = Embedding(prefix_vocab_size, PREFIX_EMBED_DIM)(prefix_input)
    suffix_embed = Embedding(suffix_vocab_size, SUFFIX_EMBED_DIM)(suffix_input)
    embed = Concatenate()([word_embed, prefix_embed, suffix_embed])
    embed_reshaped = Reshape((-1,))(embed)
    hidden1 = Dense(HIDDEN1_DIM, activation='tanh')(embed_reshaped)
    hidden2 = Dense(HIDDEN2_DIM, activation='tanh')(hidden1)
    logits = Dense(n_tags, activation='softmax')(hidden2)
    model = keras.Model(inputs=[word_input, prefix_input, suffix_input], outputs=logits)
    return model

In [16]:
word_input, prefix_input, suffix_input = Input(shape=(7,)), Input(shape=(7,)), Input(shape=(7,))
model = make_model(word_input, prefix_input, suffix_input, len(word_dict), len(prefix_dict), len(suffix_dict), len(tag_dict))

In [17]:
keras.utils.plot_model(model, 'images/model.png')

単語のID列${\bf x} = x_{-2}, x_{-1}, x, x_{+1}, x_{+2}$に対して  
$Embedding(\bf x) = [ {\bf e}_{x_{-2}}　| {\bf e}_{x_{-1}}　| {\bf e}_{x}　| {\bf e}_{x_{+1}}　|　{\bf e}_{x_{+2}} ]^T　= {\bf E}^T$,  
$ Reshape({\bf E}) = [ {\bf e}_{x_{-2}}, {\bf e}_{x_{-1}}, {\bf e}_{x}, {\bf e}_{x_{+1}}, {\bf e}_{x_{+2}} ]^T = {\bf e}$ (縦に並べる),  
$f({\bf x}) = {\mathit softmax}(W_3 \tanh (W_2 \tanh (W_1 {\bf e} + b_1) + b_2) + b_3)$.

計算グラフの可視化
<img src='images/model.png'>

In [18]:
model.summary()

__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            (None, 7)            0                                            
__________________________________________________________________________________________________
input_2 (InputLayer)            (None, 7)            0                                            
__________________________________________________________________________________________________
input_3 (InputLayer)            (None, 7)            0                                            
__________________________________________________________________________________________________
embedding_1 (Embedding)         (None, 7, 128)       2760832     input_1[0][0]                    
__________________________________________________________________________________________________
embedding_

In [19]:
model.compile(loss='categorical_crossentropy',
              optimizer=SGD(),
              metrics=['accuracy'])

In [20]:
from tensorflow.python.client import device_lib
device_lib.list_local_devices()

[name: "/device:CPU:0"
 device_type: "CPU"
 memory_limit: 268435456
 locality {
 }
 incarnation: 10640412218949148597, name: "/device:GPU:0"
 device_type: "GPU"
 memory_limit: 4948557824
 locality {
   bus_id: 1
   links {
     link {
       device_id: 1
       type: "StreamExecutor"
       strength: 1
     }
   }
 }
 incarnation: 9034008657637088813
 physical_device_desc: "device: 0, name: GeForce GTX TITAN X, pci bus id: 0000:01:00.0, compute capability: 5.2", name: "/device:GPU:1"
 device_type: "GPU"
 memory_limit: 6340041114
 locality {
   bus_id: 1
   links {
     link {
       type: "StreamExecutor"
       strength: 1
     }
   }
 }
 incarnation: 17137362309265770243
 physical_device_desc: "device: 1, name: GeForce GTX TITAN X, pci bus id: 0000:03:00.0, compute capability: 5.2"]

In [21]:
# 学習
model.fit(train_xs, train_ys, batch_size=1024, epochs=200, verbose=1, validation_data=(dev_xs, dev_ys))

# 学習がめんどい場合こっち (学習済みのパラメータを読み込み)
# model.load_weights('models/weights.h5')

Train on 929552 samples, validate on 45422 samples
Epoch 1/200
929552/929552 [==============================] - 4s 5us/step - loss: 2.4158 - acc: 0.2463 - val_loss: 2.1362 - val_acc: 0.3775
Epoch 2/200
929552/929552 [==============================] - 4s 4us/step - loss: 1.8593 - acc: 0.4542 - val_loss: 1.6301 - val_acc: 0.5015
Epoch 3/200
929552/929552 [==============================] - 4s 4us/step - loss: 1.5141 - acc: 0.5291 - val_loss: 1.3509 - val_acc: 0.5752
Epoch 4/200
929552/929552 [==============================] - 4s 4us/step - loss: 1.2493 - acc: 0.6036 - val_loss: 1.1023 - val_acc: 0.6495
Epoch 5/200
929552/929552 [==============================] - 4s 4us/step - loss: 1.0318 - acc: 0.6704 - val_loss: 0.9233 - val_acc: 0.7067
Epoch 6/200
929552/929552 [==============================] - 4s 4us/step - loss: 0.8712 - acc: 0.7250 - val_loss: 0.7886 - val_acc: 0.7525
Epoch 7/200
929552/929552 [==============================] - 4s 5us/step - loss: 0.7460 - acc: 0.7663 - val_loss: 0

929552/929552 [==============================] - 4s 5us/step - loss: 0.1369 - acc: 0.9551 - val_loss: 0.1737 - val_acc: 0.9444
Epoch 60/200
929552/929552 [==============================] - 4s 5us/step - loss: 0.1357 - acc: 0.9555 - val_loss: 0.1732 - val_acc: 0.9447
Epoch 61/200
929552/929552 [==============================] - 4s 5us/step - loss: 0.1345 - acc: 0.9559 - val_loss: 0.1732 - val_acc: 0.9448
Epoch 62/200
929552/929552 [==============================] - 4s 5us/step - loss: 0.1333 - acc: 0.9564 - val_loss: 0.1719 - val_acc: 0.9450
Epoch 63/200
929552/929552 [==============================] - 4s 5us/step - loss: 0.1322 - acc: 0.9567 - val_loss: 0.1705 - val_acc: 0.9456
Epoch 64/200
929552/929552 [==============================] - 4s 5us/step - loss: 0.1311 - acc: 0.9571 - val_loss: 0.1705 - val_acc: 0.9458
Epoch 65/200
929552/929552 [==============================] - 4s 5us/step - loss: 0.1300 - acc: 0.9574 - val_loss: 0.1695 - val_acc: 0.9462
Epoch 66/200
929552/929552 [=====

929552/929552 [==============================] - 4s 5us/step - loss: 0.0969 - acc: 0.9685 - val_loss: 0.1547 - val_acc: 0.9529
Epoch 118/200
929552/929552 [==============================] - 4s 5us/step - loss: 0.0965 - acc: 0.9687 - val_loss: 0.1542 - val_acc: 0.9532
Epoch 119/200
929552/929552 [==============================] - 4s 5us/step - loss: 0.0961 - acc: 0.9688 - val_loss: 0.1538 - val_acc: 0.9532
Epoch 120/200
929552/929552 [==============================] - 4s 5us/step - loss: 0.0957 - acc: 0.9689 - val_loss: 0.1532 - val_acc: 0.9533
Epoch 121/200
929552/929552 [==============================] - 4s 5us/step - loss: 0.0953 - acc: 0.9691 - val_loss: 0.1535 - val_acc: 0.9533
Epoch 122/200
929552/929552 [==============================] - 4s 5us/step - loss: 0.0950 - acc: 0.9691 - val_loss: 0.1552 - val_acc: 0.9525
Epoch 123/200
929552/929552 [==============================] - 4s 5us/step - loss: 0.0946 - acc: 0.9693 - val_loss: 0.1538 - val_acc: 0.9531
Epoch 124/200
929552/929552

929552/929552 [==============================] - 4s 5us/step - loss: 0.0792 - acc: 0.9740 - val_loss: 0.1545 - val_acc: 0.9542
Epoch 176/200
929552/929552 [==============================] - 4s 5us/step - loss: 0.0789 - acc: 0.9743 - val_loss: 0.1540 - val_acc: 0.9540
Epoch 177/200
929552/929552 [==============================] - 4s 5us/step - loss: 0.0787 - acc: 0.9743 - val_loss: 0.1544 - val_acc: 0.9537
Epoch 178/200
929552/929552 [==============================] - 4s 5us/step - loss: 0.0784 - acc: 0.9744 - val_loss: 0.1547 - val_acc: 0.9541
Epoch 179/200
929552/929552 [==============================] - 4s 4us/step - loss: 0.0782 - acc: 0.9744 - val_loss: 0.1551 - val_acc: 0.9541
Epoch 180/200
929552/929552 [==============================] - 4s 4us/step - loss: 0.0780 - acc: 0.9745 - val_loss: 0.1541 - val_acc: 0.9541
Epoch 181/200
929552/929552 [==============================] - 4s 4us/step - loss: 0.0777 - acc: 0.9745 - val_loss: 0.1548 - val_acc: 0.9540
Epoch 182/200
929552/929552

In [25]:
# 学習したモデルを適当に使ってみるとなんか行列が出てきます
model.evaluate(test_xs, test_ys)

55371/55371 [==============================] - 1s 25us/step


[0.12383607329298456, 0.9590218706543137]

In [26]:
# POSタグとIDの逆向きの辞書
rev_tag_dict = {v: k for k, v in tag_dict.items()}

In [50]:
# 単語リストを入力してPOSタグを予測する関数
def predict(words):
    word_ids = []
    prefix_ids = []
    suffix_ids = []
    for window in sliding_windows([PAD] * 3 + words + [PAD] * 3):
        word_ids.append([word_dict.get(word, word_dict[UNK]) for word in window])
        prefix_ids.append([prefix_dict.get(get_prefix(word), prefix_dict[UNK]) for word in window])
        suffix_ids.append([suffix_dict.get(get_suffix(word), suffix_dict[UNK]) for word in window])
    matrix = [np.array(word_ids, 'i'), np.array(suffix_ids, 'i'), np.array(suffix_ids, 'i')]
    probabilities = model.predict(matrix)
    result_ids = np.argmax(probabilities, 1)
    result = [rev_tag_dict[i] for i in result_ids]
    return result

In [51]:
predict(['this', 'is', 'a', 'test', 'sentence', '.'])

['PRON', 'VERB', 'DET', 'NOUN', 'NOUN', 'PUNCT']

In [52]:
import random
for _ in range(5):
    i = random.randint(0, len(test_sents))
    words, tags = test_sents[i]
    words = words[3:-3]
    print('sentence:\t', ' '.join(words))
    print('predict:\t', ' '.join(predict(words)))
    print('answer:\t', ' '.join(tags))

sentence:	 or is triskaidekaphobia -- fear of the number 13 -- justified ?
predict:	 CONJ VERB NOUN PUNCT NOUN ADP DET NOUN NUM PUNCT VERB PUNCT
answer:	 CONJ VERB NOUN PUNCT NOUN ADP DET NOUN NUM PUNCT VERB PUNCT
sentence:	 in addition , he said , some bigger institutional investors placed bids to buy some otc stocks whose prices were beaten down .
predict:	 ADP NOUN PUNCT PRON VERB PUNCT DET ADJ ADJ NOUN VERB NOUN PART VERB DET PROPN NOUN AUX NOUN VERB VERB ADV PUNCT
answer:	 ADP NOUN PUNCT PRON VERB PUNCT DET ADJ ADJ NOUN VERB NOUN PART VERB DET PROPN NOUN PRON NOUN AUX VERB ADP PUNCT
sentence:	 most of those freed had spent at least 25 years in prison .
predict:	 ADV ADP PRON VERB AUX VERB ADP ADJ NUM NOUN ADP NOUN PUNCT
answer:	 ADJ ADP PRON VERB AUX VERB ADP ADJ NUM NOUN ADP NOUN PUNCT
sentence:	 kenneth j. thygerson , who was named president of this thrift holding company in august , resigned , citing personal reasons .
predict:	 PROPN PROPN PROPN PUNCT PRON AUX VERB NOUN ADP DE

In [ ]:
model.save_weights('models/weights.h5')